In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, register, execute, get_backend
from qiskit.tools.visualization import circuit_drawer, plot_histogram, plot_state
from qiskit.tools.qi.qi import state_fidelity
import random, pickle, time, datetime, xlsxwriter
import scipy.io as spio
import importlib
import circuit_builder
importlib.reload(circuit_builder)
import encoding_schemes
importlib.reload(encoding_schemes)

#setup
import sys, getpass
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
    print()
except:
    APItoken = getpass.getpass('Please input your token and hit enter: ')
    qx_config = {
        "APItoken": APItoken,
        "url":"https://quantumexperience.ng.bluemix.net/api"}
    print('Qconfig.py not found in qiskit-tutorial directory; Qconfig loaded using user input.')
    
#matlab stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import matrix
from math import pi

#set 
QX_URL = "https://quantumexperience.ng.bluemix.net/api"
QX_TOKEN = "06704f99fc6dd68ab5ae2663f0861723406de8b4553cacdabdf500292c937717442daa90c2ffa19109cfb9114adb7c87711a08860d34d1da9332ac664e4e24a7"

#register token and url
register(QX_TOKEN, QX_URL)

# query for available backends
print("Available backends: ", available_backends())

backend = "ibmqx4"

def getShortest(circuits):
    circuit = min(circuits, key = len)
    return [circuit]

# code for extracting gate constructions
extract = spio.loadmat('shortnormgates.mat', squeeze_me = True)
P = extract['CNOT21norm']


# IMPORTANT SETTINGS
#
# Update the scheme to reflect your encoding scheme
scheme = encoding_schemes.Uncoded()

# Update desired outputs to reflect your circuit
# desiredOutputs = [["0000","1111"], ["0101","1010"], ["1100","0011"], ["0110","1001"]]
desiredOutputs = [["00"], ["10"], ["01"], ["11"]]

# Setup variables
initialStates = [2,3]
reps = 9

# Specify circuits
circuits = [['CNOT 2 1']]
circuits = getShortest(circuits)
# Try this one (replace CZ as appropriate) if you only want to find the desiredOutputs for a given circuit
# circuits = [CZ[0]]
#


# These update automatically with scheme
n = scheme.n
validOutputs = scheme.validOutputs

startTime = time.time()

# Main benchmarking procedure
for circuit in circuits:
    
    for compiler in initialStates:
        print("Circuit is: "+str(circuit))
        print("Compiler is: "+str(compiler))
            
        avgSuccess = 0

        for trial in range(reps):
            
            # Standard setup
            qr = QuantumRegister(n)
            cr = ClassicalRegister(n)
            qc = QuantumCircuit(qr,cr)
            
            scheme.prepareZeros(qc)
            
            # Maps the input info into actual physical qubits
            perm = scheme.perm
            scheme.setPerm()
            
            # Sets our initial state
            scheme.setInput(qc, compiler)
            
            
            # Build the circuit
            registerPerm = circuit_builder.buildCirc(qc, circuit, perm)
            
            for i in range(n):
                qc.measure(qr[registerPerm[i]],cr[i])
            
            # Standard execution code
            job = execute(qc, backend)
            #Print Job Status 
            lapse = 0
            interval = 120
            while not job.done:
                print('Status @ {} seconds'.format(interval * lapse))
                print(job.status)
                time.sleep(interval)
                lapse += 1
            print(job.status)
        
            #get Results
            result = job.result()
            data = result.get_counts(qc)
            
            
            
            
            # Prints raw data
            print(data)
            
            # Clean and process data
            successes = 0
            totalValid = 0
            for key, value in data.items():
                if key in desiredOutputs[compiler]:
                    successes += value
                if key in validOutputs:
                    totalValid += value
            
            # If we hit this, something went very wrong
            if totalValid == 0:
                print("ERROR WITH VALID INPUTS")
                
            else:
                # Comment out if running more trials
                # print("Success rate is: "+ str(successes/totalValid))
                avgSuccess += successes/totalValid
        
        # Calculate and display information
        avgSuccess /= reps
        print("Average of success rates is: "+ str(avgSuccess)+ "\n")

endTime = time.time()
print("Time running: "+str(endTime-startTime))

Qconfig loaded from /Users/Atsushi/Documents/DOMath/Jupyter/qecc_benchmark/Qconfig.py.

Available backends:  ['ibmq_16_rueschlikon', 'ibmq_5_tenerife', 'ibmq_5_yorktown', 'ibmq_qasm_simulator', 'local_qasm_simulator', 'local_statevector_simulator', 'local_unitary_simulator']
Circuit is: ['CNOT 2 1']
Compiler is: 2
Status @ 0 seconds
{'job_id': None, 'status': <JobStatus.INITIALIZING: 'job is being initialized'>, 'status_msg': 'Job is initializing. Please, wait a moment.'}
Status @ 120 seconds
{'job_id': '5b51de965c1ec600389ecbfa', 'status': <JobStatus.QUEUED: 'job is queued'>, 'status_msg': 'job is queued', 'queue_position': 3}
Status @ 240 seconds
{'job_id': '5b51de965c1ec600389ecbfa', 'status': <JobStatus.QUEUED: 'job is queued'>, 'status_msg': 'job is queued', 'queue_position': 3}


Got a 502 code response to https://quantumexperience.ng.bluemix.net/api/Jobs/5b51de965c1ec600389ecbfa?access_token=vvOGJLMGxZBVA81kMmwGJ6p0ypCeVCJ9BfA541HWVI6qDLYCSdvG5IvbnPcpA977: 502 Bad Gateway: Registered endpoint failed to handle the request.



{'job_id': '5b51de965c1ec600389ecbfa', 'status': <JobStatus.DONE: 'job has successfully run'>, 'status_msg': 'job has successfully run'}
{'00': 17, '01': 74, '10': 81, '11': 852}
Status @ 0 seconds
{'job_id': None, 'status': <JobStatus.INITIALIZING: 'job is being initialized'>, 'status_msg': 'Job is initializing. Please, wait a moment.'}
{'job_id': '5b51e07c5c1ec600389ecbfc', 'status': <JobStatus.DONE: 'job has successfully run'>, 'status_msg': 'job has successfully run'}
{'00': 20, '01': 63, '10': 101, '11': 840}
Status @ 0 seconds
{'job_id': None, 'status': <JobStatus.INITIALIZING: 'job is being initialized'>, 'status_msg': 'Job is initializing. Please, wait a moment.'}
Status @ 120 seconds
{'job_id': '5b51e10dfc7cd0003cda328d', 'status': <JobStatus.QUEUED: 'job is queued'>, 'status_msg': 'job is queued', 'queue_position': 3}
Status @ 240 seconds
{'job_id': '5b51e10dfc7cd0003cda328d', 'status': <JobStatus.QUEUED: 'job is queued'>, 'status_msg': 'job is queued', 'queue_position': 3}
S

Got a 502 code response to https://quantumexperience.ng.bluemix.net/api/Jobs/5b51eb28cb18f50037619999?access_token=vvOGJLMGxZBVA81kMmwGJ6p0ypCeVCJ9BfA541HWVI6qDLYCSdvG5IvbnPcpA977: 502 Bad Gateway: Registered endpoint failed to handle the request.



Status @ 480 seconds
{'job_id': '5b51eb28cb18f50037619999', 'status': <JobStatus.QUEUED: 'job is queued'>, 'status_msg': 'job is queued', 'queue_position': 3}
{'job_id': '5b51eb28cb18f50037619999', 'status': <JobStatus.DONE: 'job has successfully run'>, 'status_msg': 'job has successfully run'}
{'00': 92, '01': 904, '10': 9, '11': 19}
Status @ 0 seconds
{'job_id': None, 'status': <JobStatus.INITIALIZING: 'job is being initialized'>, 'status_msg': 'Job is initializing. Please, wait a moment.'}


Got a 502 code response to https://quantumexperience.ng.bluemix.net/api/Jobs/5b51edfeb91bf2003943a1a0?access_token=vvOGJLMGxZBVA81kMmwGJ6p0ypCeVCJ9BfA541HWVI6qDLYCSdvG5IvbnPcpA977: 502 Bad Gateway: Registered endpoint failed to handle the request.



Status @ 120 seconds
{'job_id': '5b51edfeb91bf2003943a1a0', 'status': <JobStatus.QUEUED: 'job is queued'>, 'status_msg': 'job is queued', 'queue_position': 3}
Status @ 240 seconds
{'job_id': '5b51edfeb91bf2003943a1a0', 'status': <JobStatus.QUEUED: 'job is queued'>, 'status_msg': 'job is queued', 'queue_position': 3}
Status @ 360 seconds
{'job_id': '5b51edfeb91bf2003943a1a0', 'status': <JobStatus.QUEUED: 'job is queued'>, 'status_msg': 'job is queued', 'queue_position': 3}
{'job_id': '5b51edfeb91bf2003943a1a0', 'status': <JobStatus.DONE: 'job has successfully run'>, 'status_msg': 'job has successfully run'}
{'00': 87, '01': 904, '10': 13, '11': 20}
Status @ 0 seconds
{'job_id': None, 'status': <JobStatus.INITIALIZING: 'job is being initialized'>, 'status_msg': 'Job is initializing. Please, wait a moment.'}
Status @ 120 seconds
{'job_id': '5b51f0601deae90038496c4e', 'status': <JobStatus.QUEUED: 'job is queued'>, 'status_msg': 'job is queued', 'queue_position': 3}
Status @ 240 seconds
{'j

Got a 502 code response to https://quantumexperience.ng.bluemix.net/api/Jobs/5b51fa4811322d003bf0fd09?access_token=vvOGJLMGxZBVA81kMmwGJ6p0ypCeVCJ9BfA541HWVI6qDLYCSdvG5IvbnPcpA977: 502 Bad Gateway: Registered endpoint failed to handle the request.



{'job_id': '5b51fa4811322d003bf0fd09', 'status': <JobStatus.DONE: 'job has successfully run'>, 'status_msg': 'job has successfully run'}
{'00': 137, '01': 856, '10': 14, '11': 17}
Status @ 0 seconds
{'job_id': None, 'status': <JobStatus.INITIALIZING: 'job is being initialized'>, 'status_msg': 'Job is initializing. Please, wait a moment.'}
Status @ 120 seconds
{'job_id': '5b51fc2e1deae90038496c7a', 'status': <JobStatus.QUEUED: 'job is queued'>, 'status_msg': 'job is queued', 'queue_position': 3}
{'job_id': '5b51fc2e1deae90038496c7a', 'status': <JobStatus.DONE: 'job has successfully run'>, 'status_msg': 'job has successfully run'}
{'00': 156, '01': 847, '10': 9, '11': 12}
Status @ 0 seconds
{'job_id': None, 'status': <JobStatus.INITIALIZING: 'job is being initialized'>, 'status_msg': 'Job is initializing. Please, wait a moment.'}
Status @ 120 seconds
{'job_id': '5b51fd20a90c3f003f7e5199', 'status': <JobStatus.QUEUED: 'job is queued'>, 'status_msg': 'job is queued', 'queue_position': 3}
S